In [66]:
import pandas as pd
from datetime import datetime
from pulp import LpMaximize, LpProblem, LpVariable, lpSum


In [67]:
df = pd.read_csv(f"../outputs/lineup_{datetime.now().date()}.csv")

In [68]:
df.head()

,Unnamed: 0,ID,First Name,Last Name,Position,Team,Opponent,Game,Time,Salary,...,Opponent_Modifier,Team_Modifier,Pace_Modifier,Team_Odds,Opponent_Odds,Adjusted_Team_Odds,Adjusted_Opponent_Odds,Normalized_Team_Odds,Normalized_Opponent_Odds,Value_Score
0,0,nba.p.4245,Al,Horford,PF,BOS,IND,BOS@IND,7:00PM EDT,21,...,1.0,1.2715,0.245,1.4,3.055556,0.714286,0.327273,0.802381,0.367636,0.396023
1,1,nba.p.4622,Jrue,Holiday,PG,BOS,IND,BOS@IND,7:00PM EDT,26,...,1.0,1.2715,0.245,1.4,3.055556,0.714286,0.327273,0.802381,0.367636,0.297115
2,2,nba.p.5464,Kristaps,Porzingis,C,BOS,IND,BOS@IND,7:00PM EDT,34,...,1.0,1.2715,0.245,1.4,3.055556,0.714286,0.327273,0.802381,0.367636,NaN
3,3,nba.p.5602,Jaylen,Brown,SF,BOS,IND,BOS@IND,7:00PM EDT,37,...,1.0,1.2715,0.245,1.4,3.055556,0.714286,0.327273,0.802381,0.367636,0.411500
4,4,nba.p.5765,Jayson,Tatum,PF,BOS,IND,BOS@IND,7:00PM EDT,51,...,1.0,1.2715,0.245,1.4,3.055556,0.714286,0.327273,0.802381,0.367636,0.324879


## Evaluate Player Value

In [62]:
# Create LP Problem
model = LpProblem(name="optimal-fantasy-team", sense=LpMaximize)

# Create LpVariable for each player (0 or 1 to indicate whether the player is selected)
players = df['ID'].tolist()
player_vars = LpVariable.dicts("Player", players, cat="Binary")

# Objective Function: Maximize the sum of Average_FPS and Value_Score
model += lpSum((df.loc[df['ID'] == player, 'weighted_FPS'].values[0] + df.loc[df['ID'] == player, 'Value_Score'].values[0]) * player_vars[player] for player in players)

# Constraint 1: Total Salary should be less than or equal to 200
model += lpSum(df.loc[df['ID'] == player, 'Salary'].values[0] * player_vars[player] for player in players) <= 200


In [22]:
model.solve()

1

## Top 5 Value per position

In [ ]:
## Select Player

positions = {'PG', 'SG', 'G,' 'SF', 'PF', 'F', 'C', 'UTIL'}

# make a dictionary where position is key and yahoo player ID is the value.
lineup = dict.fromkeys(positions)